O objetivo desse notebook é a construção de um modelo capaz de predizer o valor do IBU (*International Bitterness Units*), que mensura o armagor, para um conjunto de cervejas.

Os dados utilizados no treinamento foram coletados da [PUNK API](https://punkapi.com/documentation/v2) e posteriormente tratados e armazenados em buckets S3 na AWS.

Para utilizá-los nesse notebook, será feito uma query SQL via Athena na tabela Glue criada na AWS, que retornará os dados coletados em um certo período de tempo da API.

# Bibliotecas

In [3]:
import os
import dotenv
import pickle
import time

import boto3
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import requests

# Leitura dos dados

In [62]:
from pyathena import connect
dotenv.load_dotenv(".env")


S3_OUTPUT = "s3://punkapi-data-from-glue"
AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]
REGION_NAME = os.environ["REGION_NAME"]


conn = connect(aws_access_key_id=AWS_ACCESS_KEY_ID,
                 aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                 s3_staging_dir=S3_OUTPUT,
                 region_name=REGION_NAME)

query_string = (
    "SELECT TRY_CAST(abv as DOUBLE) as abv, " \
    "TRY_CAST(ebc as INTEGER) as ebc, " \
    "TRY_CAST(ibu as DOUBLE) as ibu, " \
    "TRY_CAST(id as BIGINT) as id, " \
    "TRY_CAST(name as VARCHAR) as name, " \
    "TRY_CAST(ph as DOUBLE) as ph, " \
    "TRY_CAST(srm as DOUBLE) as srm, " \
    "TRY_CAST(target_fg as BIGINT) as target_fg, " \
    "TRY_CAST(target_og as DOUBLE) as target_og " \
    "FROM punk_api_database.cleaned_data_from_punkapi"
)
df = pd.read_sql(query_string, conn)

In [63]:
df

,abv,ebc,ibu,id,name,ph,srm,target_fg,target_og
0,41.0,40,1085.0,137,Sink The Bismarck!,4.4,20.00,1016.0,1085.0
1,5.4,12,35.0,67,Hunter Foundation Pale Ale,4.4,5.75,1008.0,1050.0
2,7.5,30,70.0,89,Citra,4.4,15.00,1013.0,1068.0
3,4.5,25,55.0,111,Vagabond Pilsner,4.4,12.50,1012.0,1046.0
4,9.5,250,85.0,56,Black Eyed King Imp,4.4,125.00,1022.0,1095.0
...,...,...,...,...,...,...,...,...,...
269,16.1,400,85.0,177,Dog D,4.3,200.00,1015.0,1125.0
270,9.2,20,149.0,57,Prototype 27,4.4,9.80,1014.0,1083.0
271,6.2,35,50.0,104,India Pale Weizen (w/ Weihenstephan),4.0,17.50,1010.0,1056.0
272,11.3,164,50.0,150,AB:13,4.4,83.00,1020.0,1098.0


# Pré-processamento do dado

Informações sobre o dado:

* ABV (Alcohol By Volume) - Quantidade de álcool presente na cerveja.
* EBC (European Brewery Convention) - Medida técnica europeia para a cor da cerveja.
* IBU (International Bitterness Units) - Índice que mede o amargor da cerveja.
* pH - Escala para determinar acidez/alcalinidade de uma solução.
* SRM (Standard Reference Method) - Sistema para medição de cor da cerveja usualmente aplicado nos Estados Unidos.
* Target FG (Final Gravity) - A gravidade final (FG) é a quantidade de açúcar que sobra quando a fermentação termina.
* Target OG (Original Gravity) - A gravidade original (OG) mede quanto açúcar está presente no mosto antes de ser fermentado. 

## Remoção de colunas

Duas colunas serão removidas do dado coletado: "id" e "name". A primeira, por se tratar apenas do identificador único de cada cerveja, não representa uma característica útil para determinar os IBU's. A segunda, apesar de poder ser transformada para um conjunto de valores numéricos via vetorização, será removida para simplificar o processo de treinamento.

In [64]:
INTEREST_COLUMNS = ["abv", "ebc", "ibu", "ph", "srm", "target_fg", "target_og"]
df_cleaned = df[INTEREST_COLUMNS]

In [65]:
df_cleaned

,abv,ebc,ibu,ph,srm,target_fg,target_og
0,41.0,40,1085.0,4.4,20.00,1016.0,1085.0
1,5.4,12,35.0,4.4,5.75,1008.0,1050.0
2,7.5,30,70.0,4.4,15.00,1013.0,1068.0
3,4.5,25,55.0,4.4,12.50,1012.0,1046.0
4,9.5,250,85.0,4.4,125.00,1022.0,1095.0
...,...,...,...,...,...,...,...
269,16.1,400,85.0,4.3,200.00,1015.0,1125.0
270,9.2,20,149.0,4.4,9.80,1014.0,1083.0
271,6.2,35,50.0,4.0,17.50,1010.0,1056.0
272,11.3,164,50.0,4.4,83.00,1020.0,1098.0


## Ajuste dos tipos dos valores

Para evitar exceções durante o treinamento do modelo, é importante que todas colunas contenham apenas valores numéricos. Como pode se ver abaixo, esse não é o caso para algumas colunas:

In [66]:
df_cleaned.dtypes

abv          float64
ebc            int64
ibu          float64
ph           float64
srm          float64
target_fg    float64
target_og    float64
dtype: object

In [67]:
df_cleaned = df_cleaned.apply(pd.to_numeric)

In [68]:
df_cleaned.dtypes

abv          float64
ebc            int64
ibu          float64
ph           float64
srm          float64
target_fg    float64
target_og    float64
dtype: object

## Remover linhas com NaN's

Também é necessário remover ou substituir linhas que possuem NaN's, pois são valores faltantes e podem impactar o treinamento dos modelos. Vamos checar quantos casos desses o dataset possui:

In [69]:
df_cleaned[df_cleaned.isna().any(axis=1)]

,abv,ebc,ibu,ph,srm,target_fg,target_og
15,6.7,15,40.0,NaN,NaN,NaN,NaN
26,6.7,15,40.0,NaN,NaN,NaN,NaN
54,5.0,110,50.0,NaN,NaN,NaN,NaN
136,18.3,15,50.0,NaN,NaN,NaN,NaN
249,7.5,200,90.0,NaN,NaN,NaN,NaN


Como é uma quantidade mínima de casos em relação ao conjunto total, vamos apenas remover essas linhas:

In [70]:
df_cleaned.dropna(inplace=True)

## Feature scaling

Para auxiliar o treinamento e melhorar a performance do modelo, é importante que a escala dos valores presentes no dado de treinamento esteja contida dentro de determinada faixa. Dessa forma, será usado uma normalização min-max que manterá os valores entre 0 e 1. Ela será incluída em uma pipeline do scikit-learn, juntamente com o modelo em si. Abaixo, uma amostra do que acontece ao aplicar a estratégia no dado coletado:

In [71]:
scaler = MinMaxScaler()
scaled_data = df_cleaned.copy()
scaled_data[INTEREST_COLUMNS] = scaler.fit_transform(df_cleaned)

In [72]:
scaled_data

,abv,ebc,ibu,ph,srm,target_fg,target_og
0,1.000000,0.066667,1.000000,0.055696,0.065574,0.903111,0.943478
1,0.120988,0.020000,0.032258,0.055696,0.018852,0.896000,0.913043
2,0.172840,0.050000,0.064516,0.055696,0.049180,0.900444,0.928696
3,0.098765,0.041667,0.050691,0.055696,0.040984,0.899556,0.909565
4,0.222222,0.416667,0.078341,0.055696,0.409836,0.908444,0.952174
...,...,...,...,...,...,...,...
269,0.385185,0.666667,0.078341,0.054430,0.655738,0.902222,0.978261
270,0.214815,0.033333,0.137327,0.055696,0.032131,0.901333,0.941739
271,0.140741,0.058333,0.046083,0.050633,0.057377,0.897778,0.918261
272,0.266667,0.273333,0.046083,0.055696,0.272131,0.906667,0.954783


# Treinamento do modelo

Para realizar a tarefa de predizer dos valores de IBU, serão treinado um model de regressão linear.

In [73]:
X_COLUMNS = ["abv", "ebc", "ph", "srm", "target_fg", "target_og"]
Y_COLUMN = "ibu"

In [74]:
X, y = df_cleaned[X_COLUMNS], df_cleaned[Y_COLUMN]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [75]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([('scaler', MinMaxScaler()), ('regressor', LinearRegression())])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

In [76]:
print(f"R2 (Reg. Linear): {r2_score(y_test, y_pred)}")

R2 (Reg. Linear): -292.2252628515341


# Armazenamento do modelo em um bucket S3

É possível acessar e realizar predições de maneira remota utilizando a infraestrutura da AWS. Para isso, o modelo treinado será armazendo em um bucket S3 da AWS e consumido via uma função lambda, acessada via API Gateway.


In [77]:
BUCKET_NAME = "beers-linear-regressor"
FILENAME = "pipeline.pkl"
FILE_PATH = os.path.join("/tmp/", FILENAME)

In [78]:
session = boto3.Session(aws_access_key_id=AWS_ACCESS_KEY_ID,
                        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                        region_name=REGION_NAME)

s3_client = session.client("s3")

with open(FILE_PATH, "wb") as f:
    pickle.dump(pipe, f)

s3_client.upload_file(FILE_PATH, BUCKET_NAME, FILENAME)

# Predições via lambda 

In [79]:
some_beers = requests.get(f"https://api.punkapi.com/v2/beers/?ids=42|77")

In [90]:
lambda_client = session.client('lambda')
response = lambda_client.invoke(
    FunctionName='predict_ibu',
    LogType='None',
    Payload=json.dumps({"data": some_beers.json()})
)

In [91]:
response

{'ResponseMetadata': {'RequestId': '21548c3a-d831-467a-9a69-549255d6d273',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 21 Oct 2021 15:08:19 GMT',
   'content-type': 'application/json',
   'content-length': '97',
   'connection': 'keep-alive',
   'x-amzn-requestid': '21548c3a-d831-467a-9a69-549255d6d273',
   'x-amzn-remapped-content-length': '0',
   'x-amz-executed-version': '$LATEST',
   'x-amzn-trace-id': 'root=1-61718263-4978e47f6cbc2bfe55c10f16;sampled=0'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'ExecutedVersion': '$LATEST',
 'Payload': <botocore.response.StreamingBody at 0x7feff1b24a90>}

In [92]:
res_json = json.loads(response['Payload'].read().decode("utf-8"))

In [93]:
res_json

{'statusCode': 200,
 'predictions': '[{"42": 90.42234637966311}, {"77": 10.191280082473838}]'}